In [457]:
from llm_model import CienaGPTChatModel
import pydantic
import os
import requests
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup

In [458]:
class BPAHelpPage(BaseModel):
    header: str = Field(description="Header or Title of the page")
    #p: List[str] = Field(description="List of values extracted from p tags. P tags can also be inside of div tags")
    #lists: Mapping[str, List[str]] = Field(description="Dictionary of either ol or ul tags. The key will be the p value before the element while the values are the values of the li tags under them.")
    tables: List[str] = Field(description="List of table tags as github markdown. ")



In [459]:
output_parser = PydanticOutputParser(pydantic_object = BPAHelpPage)
format_instructions = output_parser.get_format_instructions()

In [470]:
human_text = "{input}"
messages = [
    SystemMessage(
        content='Extract all table tags from the provided html user input. When a table is embedded in another table, extract it as a text instead of a separate table. If there are rows that do not fit in the structure of the table, conform them to the structure of the table. Use Github markdown for output. ' + format_instructions
    ),
    HumanMessagePromptTemplate.from_template(human_text)
]
prompt = ChatPromptTemplate.from_messages(messages)

In [471]:
llm = CienaGPTChatModel(model_name='test-model', model_id = '5', base_address='https://gptapi.ciena.com/', okta_domain='ciena.okta.com', okta_custom_scope='api', client_secret='uNU1c9Vl01WfkBdQDUivZr0O1pR3qxJpe5js3d1PJhwqGO63WEvqvOXgBux9dRyi', client_id='0oa1x6yhddqyoeoTY0h8')

In [472]:
chain = prompt | llm | output_parser

In [473]:
with open('uaa-working-with-slas.html') as f:
    input_text = f.read()
print(input_text)
soup = BeautifulSoup(input_text, "html.parser")
html_input = soup.body.text


<!DOCTYPE html>
<html xmlns:MadCap="http://www.madcapsoftware.com/Schemas/MadCap.xsd" lang="en-us" xml:lang="en-us" class="_Skins_HTML5___Side_Navigation_Skin" data-mc-search-type="Stem" data-mc-help-system-file-name="Default.xml" data-mc-path-to-help-system="../../" data-mc-has-content-body="True" data-mc-toc-path="[%=System.LinkedTitle%]|SLAs" data-mc-target-type="WebHelp2" data-mc-runtime-file-type="Topic;Default" data-mc-preload-images="false" data-mc-in-preview-mode="false">
<head>
    <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
    <meta charset="utf-8"/>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
    <meta name="msapplication-config" content="../../Skins/Favicons/browserconfig.xml"/>
    <link rel="shortcut icon" href="../../Skins/Favicons/favicon-32x32.png"/>
    <link rel="icon" sizes="96x96" href="../../Skins/Favicons/favicon-96x96.png"/>
    <link rel="icon" sizes="32x32" href="../../Skins/Favicons/favicon-32x32.png"/

In [474]:
elements = chain.invoke({"input":html_input})

success: 200
{'result': {'id': 0, 'oktaUserId': None, 'openAiId': 'chatcmpl-ACCMioMBtZ6zf2apg4DEoM0XD7kCB', 'model': 'gpt-4o-2024-05-13', 'title': None, 'choices': [{'message': {'content': '```json\n{\n  "header": "Working with SLAs",\n  "tables": [\n    "| **Name** | **Description** |\\n| --- | --- |\\n| Name | Enter a name for the SLA. |\\n| Conformance Period | Select the Conformance Period from the drop-down list - Daily, Weekly, Monthly, Quarterly, Half yearly, Yearly. It is the period, for which the data collected will be consolidated and SLA conformance will be calculated. For example, if you select Daily, then the data collected in a day will be consolidated and conformance will be calculated for the day. |\\n| Domain | Select a Domain from the drop-down list. See Administrative Domains to know more about domains. |\\n| Start Date | Select the start date from when the SLA needs to be active for the service type. |\\n| Active Time | The period during which the SLA should be acti

In [475]:
print(len(elements.tables))

2


In [479]:
print(elements.tables[1])

| **Name** | **Description** |
| --- | --- |
| Name | Enter a Name for the conformance objective. |
| Conformance Type | Select the Conformance Type from the drop-down list - Conformance %, Avg, Sum, Min, Max. |
| Conformance State | Select the High and Low values for Violated - Conformance State. |
| Conformance Condition | Add appropriate Conformed state and Warning state values for Target % value Property. Select (Warning, Minor, Major, Critical) from the Conformed state drop-down and Warning state drop-down for Alarm Severity Property. For Conformance Type except Conformance %, you need to add at least one of the Value High or Value Low for Conformed state and Warning state. Make sure that the values of Conformance / Warning - Value High and Value low follows the given order - Conformance State high >Waring state high >Warning State low >Conformance State low. |
| Formula | Define the formula used to compute the conformance value using the metric variables defined in the Metric Var

In [477]:
elements.header

'Working with SLAs'